In [ ]:
import qcodes as qc
from zhinst.qcodes import ZISession
import h5py
station = qc.Station()

device_id = "DEV5149"  # 実際のデバイスIDに置き換えてください
server_host = "169.254.254.194"  # 実際のサーバーホストに置き換えてください
session = ZISession(server_host)
MFLI_lockin = session.connect_device(device_id)
station.add_component(MFLI_lockin)

In [ ]:
OUT_CHANNEL = 0
OUT_MIXER_CHANNEL = 1 # UHFLI: 3, HF2LI: 6, MFLI: 1
IN_CHANNEL = 0
DEMOD_INDEX = 0
OSC_INDEX = 0

with MFLI_lockin.set_transaction():
    # MFLI_lockin.sigins[IN_CHANNEL].float(0) #grounded=0
    # MFLI_lockin.sigins[IN_CHANNEL].diff(1) #diff=1
    MFLI_lockin.sigins[IN_CHANNEL].ac(0)
    # MFLI_lockin.sigins[IN_CHANNEL].arange(0.2)
    MFLI_lockin.sigins[IN_CHANNEL].autorange(1) #on=1

    MFLI_lockin.demods[DEMOD_INDEX].enable(True)
    MFLI_lockin.demods[DEMOD_INDEX].rate(10e3)
    MFLI_lockin.demods[DEMOD_INDEX].adcselect(IN_CHANNEL)
    MFLI_lockin.demods[DEMOD_INDEX].timeconstant(0.01)
    MFLI_lockin.demods[DEMOD_INDEX].oscselect(OSC_INDEX)
    MFLI_lockin.demods[DEMOD_INDEX].harmonic(1)
    # MFLI_lockin.demods[DEMOD_INDEX].phaseshift(0)
    # MFLI_lockin.demods[DEMOD_INDEX].order(3)
    # MFLI_lockin.demods[DEMOD_INDEX].sinc(1)
    # MFLI_lockin.demods[DEMOD_INDEX].enable(1)

    MFLI_lockin.sigouts[OUT_CHANNEL].on(False)
    MFLI_lockin.sigouts[OUT_CHANNEL].enables[OUT_MIXER_CHANNEL].value(1)
    # MFLI_lockin.sigouts[OUT_CHANNEL].range(1)
    # MFLI_lockin.sigouts[OUT_CHANNEL].amplitudes[0].value(OUT_MIXER_CHANNEL)
    MFLI_lockin.sigouts[OUT_CHANNEL].amplitudes[1].value(2e-3)
    # MFLI_lockin.sigouts[OUT_CHANNEL].amplitudes[2].value(OUT_MIXER_CHANNEL)
    # MFLI_lockin.sigouts[OUT_CHANNEL].amplitudes[3].value(OUT_MIXER_CHANNEL)
    MFLI_lockin.sigouts[OUT_CHANNEL].autorange(0)


In [ ]:
LOOPCOUNT = 1

sweeper = session.modules.sweeper
sweeper.device(MFLI_lockin)

sweeper.gridnode(MFLI_lockin.oscs[OSC_INDEX].freq)
sweeper.start(4e3)
sweeper.stop(500e3) # 500e3 for MF MFLI_lockins, 50e6 for others
sweeper.samplecount(100)
sweeper.xmapping(1)
sweeper.bandwidthcontrol(2)
sweeper.bandwidthoverlap(0)
sweeper.scan(0)
sweeper.loopcount(LOOPCOUNT)
sweeper.settling.time(0)
sweeper.settling.inaccuracy(0.001)
sweeper.averaging.tc(10)
sweeper.averaging.sample(10)

In [ ]:
sample_node = MFLI_lockin.demods[DEMOD_INDEX].sample
sweeper.subscribe(sample_node)

In [ ]:
sample_node()

In [ ]:
sweeper.save.fileformat.node_info.options

In [ ]:
sweeper.save.filename('sweep_with_save')
sweeper.save.fileformat('hdf5')

In [ ]:
import logging
import sys

handler = logging.StreamHandler(sys.stdout)
logging.getLogger("zhinst.toolkit").setLevel(logging.INFO)
logging.getLogger("zhinst.toolkit").addHandler(handler)
sweeper.save.saveonread(True)

In [ ]:
sweeper.execute()
sweeper.read()
print(f"Perform {LOOPCOUNT} sweeps")
sweeper.wait_done(timeout=300)

In [ ]:
sweeper.save.save(True)
# Wait until the save is complete. The saving is done asynchronously in the background
# so we need to wait until it is complete. In the case of the sweeper it is important
# to wait for completion before before performing the module read command. The sweeper has
# a special fast read command which could otherwise be executed before the saving has
# started.
sweeper.save.save.wait_for_state_change(True, invert=True, timeout=5)

In [ ]:
sweeper.unsubscribe(sample_node)
data = sweeper.read()

In [ ]:
print(data[sample_node][0][0]['frequency'])

In [ ]:
type(data)

In [ ]:
num_sweeps = len(data[sample_node])
assert num_sweeps == LOOPCOUNT, (
    f"The sweeper returned an unexpected number of sweeps: "
    f"{num_sweeps}. Expected: {LOOPCOUNT}."
)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

node_samples = data[sample_node]

_, (ax1, ax2) = plt.subplots(2, 1)
for sample in node_samples:
    frequency = sample[0]["frequency"]
    demod_r = np.abs(sample[0]["x"] + 1j * sample[0]["y"])
    phi = np.angle(sample[0]["x"] + 1j * sample[0]["y"])
    ax1.plot(frequency, demod_r)
    ax2.plot(frequency, phi)
ax1.set_title(f"Results of {len(node_samples)} sweeps.")
ax1.grid()
ax1.set_ylabel(r"Demodulator R ($V_\mathrm{RMS}$)")
ax1.set_xscale("log")
ax1.autoscale()

ax2.grid()
ax2.set_xlabel("Frequency ($Hz$)")
ax2.set_ylabel(r"Demodulator Phi (radians)")
ax2.set_xscale("log")
ax2.autoscale()

plt.draw()
plt.show()

#Live Plot Test

In [ ]:
import time
import os
import qcodes as qc

import numpy as np
from plottr.data.datadict_storage import DataDict, DDH5Writer
from tqdm import tqdm

In [ ]:
import logging
import sys

handler = logging.StreamHandler(sys.stdout)
logging.getLogger("zhinst.toolkit").setLevel(logging.INFO)
logging.getLogger("zhinst.toolkit").addHandler(handler)

sample_node = MFLI_lockin.demods[DEMOD_INDEX].sample
sweeper.subscribe(sample_node)

In [ ]:
data_path = 'C:\\Users\\qipe\\annealing\\data'
timestr = time.strftime("%Y%m%d-%H%M%S")
measurement_name = timestr

In [ ]:
data_dict = DataDict(
    freq = dict(unit ='Hz'),
    amp = dict(unit = 'V', axes = ["freq"])
)
data_dict.validate

In [ ]:
def freq_sweep():
    MFLI_lockin.sigouts[OUT_CHANNEL].on(True)
    sweeper.save.saveonread(True)
    sweeper.execute()
    print(f"Perform {LOOPCOUNT} sweeps")
    try:
        while sweeper.progress() < 1.0:
            print(f"progress {sweeper.progress()*100}%")
            data = sweeper.read()
            time.sleep(0.1)
    finally:
        data = sweeper.read()
        MFLI_lockin.sigouts[OUT_CHANNEL].on(False)
        sweeper.unsubscribe(sample_node)
        return data
        # sweeper.save.save(True)
        # # Wait until the save is complete. The saving is done asynchronously in the background
        # # so we need to wait until it is complete. In the case of the sweeper it is important
        # # to wait for completion before before performing the module read command. The sweeper has
        # # a special fast read command which could otherwise be executed before the saving has
        # # started.
        # sweeper.save.save.wait_for_state_change(True, invert=True, timeout=5)
 

In [ ]:
with DDH5Writer(data_dict, data_path, name = measurement_name) as writer:
    data = freq_sweep()
    writer.add_data(
        freq = data[sample_node][0][0]["frequency"],
        amp = data[sample_node][0][0]["r"]
    )

In [ ]:
data[sample_node][0][0]["r"]

In [ ]:
data[sample_node][0][0]["frequency"]